# OpenHPC v2 テンプレートによる機械学習システムの構築

---

## 概要

VCP SDKを用いてクラウド上に GPU を持つ仮想サーバを作成し、OpenHPC 環境の構築を行います。仮想サーバでは TensorFlow コンテナを動作させます。

![構成](images/ohpc-000.png)

## VCP の初期化
次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンで VCP を初期化します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## パラメータ設定（based on 010-パラメータ設定.ipynb）

必要に応じパラメータを変更してください。

In [ ]:
# ★★★ VC利用者情報 ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
ssh_public_key_path = '~/.ssh/id_rsa.pub'  # VC 利用者：ssh 公開鍵
ssh_private_key_path = '~/.ssh/id_rsa'     # VC 利用書：ssh 秘密鍵


# ★★★ 資源情報 ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# ＝＝＝ プロバイダ ＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
vc_provider = 'aws'                        # 利用プロバイダ

# ＝＝＝ UnitGroup 名 ＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
ugroup_name = 'TFclusterGPU'

# ＝＝＝ 計算ノード ＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
compute_flavor = 'gpu'                     # 資源タイプ：flavor で指定する場合
compute_instance_type = 'g3s.xlarge'       # 資源タイプ：具体的なインスタンス名で指定する場合
compute_nodes = 1                          # 計算ノード数
compute_root_size = 32                     # 計算ノードの root ディスクサイズ(GB、32GB 以上)
compute_use_gpu = True                     # True: GPU 使用、False: GPU 不使用
c_hostname_prefix = 'c'                    # 計算ノードのホスト名のプレフィックス
c_ip_address = '172.30.2.121'              # 計算ノード群の先頭の IP アドレス（他の資源とアドレスが重ならないこと、計算ノード数分の連番が必要）

# ＝＝＝ マスターノード ＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
master_flavor = 'small'                    # 資源タイプ：flavor で指定する場合
# master_instance_type = 'インスタンス名'     # 資源タイプ：具体的なインスタンス名で指定する場合
master_root_size = 60                      # マスターノードの root ディスクサイズ(GB、60GB 以上))
master_hostname = 'master'                 # マスターノードのホスト名
master_ipaddress = '172.30.2.120'          # マスターノードの IP アドレス（他の資源とアドレスが重ならないこと）
 
# ＝＝＝ NFS 用ディスク（マスターノードに接続） ＝＝＝＝＝＝ ※ NFS ディスクを作成しない場合は nfs_disk_size をコメントにしてください
nfs_disk_size = 64                         # NFS 用ディスクのサイズ(GB、16GB 以上）
nfs_device = '/dev/xvdf'                   # デバイス名


# ★★★ OpenHPC 環境情報 ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# ＝＝＝ ユーザ情報（別テンプレートで追加・削除可能） ＝＝＝
ohpc_user = 'user00'                       # OpenHPC 環境ユーザ
ohpc_user_pubkey = '~/.ssh/id_rsa.pub'     # OpenHPC user: ssh 公開鍵
ohpc_user_prvkey = '~/.ssh/id_rsa'         # OpenHPC user: ssh 秘密鍵

### ユーザー設定パラメータの値のヒント表示（必要に応じて実行してください）

UnitGroup 名は、VCP SDKで作成した他の環境と区別するために既存の UnitGroup 名と異なる名前とします。次のセルを実行すると既存の UnitGroup 名が表示されます。

In [ ]:
vcp.df_ugroups()

次のセルを実行すると flavor と 上のセルで指定した vc_provider のインスタンス名の対応が表示されます。

In [ ]:
vcp.df_flavors(vc_provider)            

次のセルを実行すると使用可能な IP アドレスレンジが表示されます。IP アドレスを指定する場合は、重複しないように注意してください。

In [ ]:
print(vcp.get_vpn_catalog(vc_provider).get('private_network_ipmask'))  # IP アドレスレンジ

## 構築

### 準備

内部パラメータの設定や編集を行い、関連 notebook と共有するために groups_vars フォルダに保存します。ファイル名は {UnitGroup名}.yml です。

構築開始日時

In [ ]:
!date

#### 計算ノード hosts 情報作成（based on 010-パラメータ設定.ipynb）

計算ノードのIPアドレスとホスト名のdict型変数を作成します。

In [ ]:
import ipaddress

c_addr = ipaddress.ip_address(c_ip_address)
compute_etc_hosts = dict([
    (f'{c_addr + x}', f'{c_hostname_prefix}{x + 1}')
    for x in range(compute_nodes)   
])

import json
print(json.dumps(compute_etc_hosts, indent=2))

#### Slurm パラメータ作成（based on 010-パラメータ設定.ipynb）

Slurmに関連するパラメータを指定します。

![etc_hosts](images/ohpc-006.png)

=== munge.key =====================================================================================================

MUNGEはHPCクラスタ環境のための認証サービスです。この節ではSLURMがコンポーネント間の認証に利用するMUNGEの鍵ファイルmunge.keyを作成します。

munge.key に書き込む内容を乱数から生成します。

In [ ]:
import secrets

munge_key = secrets.token_bytes(1024)

`munge.key` の内容は秘匿情報になるので、`group_vars`ファイルではなく VC Controller の HashiCorp Vault に保存します。
HashiCorp Vault は秘密情報を保存するための Key Valueストアです。保持する情報は暗号化されます。


HashiCorp Valutのなかの記録場所となるパスを次のセルで指定します。

In [ ]:
vault_path_munge_key = f'cubbyhole/OpenHPC/{ugroup_name}/munge.key'
print(vault_path_munge_key)

=== slurm.conf ====================================================================================================

`slurm.conf` に登録する計算ノードのホスト名を指定します。

計算ノードとして利用するホスト名を `slurm.conf` に登録する必要があります。
次のセルを実行すると「[5.2  計算ノードのIPアドレスとホスト名](#計算ノードのIPアドレスとホスト名)」で指定した値に従い、`NodeName` の値を設定します。

> ただしIPアドレスに任意の値を指定した場合は、次のセルを実行するとエラーとなることがあります。その場合、指定したホスト名に合わせて設定を行ってください。

In [ ]:
slurm_conf = {
    'NodeName': f'{c_hostname_prefix}[1-{compute_nodes}]' if compute_nodes > 1 else f'{c_hostname_prefix}1',
}

`slurm_conf`に設定した値を確認します。

In [ ]:
import json
print(json.dumps(slurm_conf, indent=2))

#### パラメータの保存（based on 010-パラメータ設定.ipynb）

=== munge.key の保存 ===============================================================================================

`munge.key`は秘匿情報のため、暗号化され記録される HashiCorp Vaultに保存します。

> 保存に成功すると `<Response [204]>` と表示されます。

In [ ]:
import requests
import base64

payload = {
    'munge.key': base64.b64encode(munge_key).decode('UTF-8'),
}

vault_url = f'{vcp.vcc_info()["vault_url"]}/v1/{vault_path_munge_key}'

custom_headers = {
    'X-Vault-Token': vcc_access_token,
}

r = requests.post(vault_url, headers=custom_headers, json=payload)
r

=== その他のパラメータの保存 ==========================================================================================

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookでは `group_vars/` にあるファイルを `group_vars ファイル`と呼ぶことにします。

値の保存を行う前に、入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

> IPアドレスに到達可能なものがあるかをチェックするので、ある程度時間が掛かります。

In [ ]:
%run scripts/utils_addition.py
from pathlib import Path

if 'nfs_disk_size' in vars():
    check_parameters(
        _params=dict(vc_provider=vc_provider),
        nfs_disk_size=nfs_disk_size,
        nfs_device=nfs_device,
    )

check_parameters(
    _params=dict(
        vcp=vcp,
        vc_provider=vc_provider,
        compute_nodes=compute_nodes,
        nfs_disk_size=nfs_disk_size if 'nfs_disk_size' in vars() else 0,
    ),
    vc_provider=vc_provider,
    ugroup_name=ugroup_name,
    compute_flavor=compute_flavor,
    compute_nodes=compute_nodes,
    compute_etc_hosts=compute_etc_hosts,
    c_ip_address=c_ip_address,
    master_flavor=master_flavor,
    master_root_size=master_root_size,
    master_hostname=master_hostname,
    master_ipaddress=master_ipaddress,
    ssh_public_key_path=str(Path(ssh_public_key_path).expanduser()),
    ssh_private_key_path=str(Path(ssh_private_key_path).expanduser()),
    ohpc_user_pubkey=str(Path(ohpc_user_pubkey).expanduser()),
    ohpc_user_prvkey=str(Path(ohpc_user_prvkey).expanduser()),
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    ugroup_name,
    ugroup_name=ugroup_name,
    vc_provider=vc_provider,
    compute_flavor=compute_flavor,
    compute_nodes=compute_nodes,
    compute_etc_hosts=compute_etc_hosts,
    compute_use_gpu=compute_use_gpu,
    c_ip_address=c_ip_address,
    master_flavor=master_flavor,
    master_root_size=master_root_size,
    master_hostname=master_hostname,
    master_ipaddress=master_ipaddress,
    ssh_public_key_path=str(Path(ssh_public_key_path).expanduser()),
    ssh_private_key_path=str(Path(ssh_private_key_path).expanduser()),
    ohpc_user=ohpc_user,
    ohpc_user_pubkey=str(Path(ohpc_user_pubkey).expanduser()),
    ohpc_user_prvkey=str(Path(ohpc_user_prvkey).expanduser()),
    vault_path_munge_key=vault_path_munge_key,
    slurm_conf=slurm_conf,
)

if 'compute_instance_type' in vars():
    update_group_vars(
        ugroup_name, 
        compute_instance_type=compute_instance_type,
    )

if 'compute_root_size' in vars():
    update_group_vars(
        ugroup_name, 
        compute_root_size=compute_root_size,
    )
    
if 'master_instance_type' in vars():
    update_group_vars(
        ugroup_name, 
        master_instance_type=master_instance_type,
    )

if 'nfs_disk_size' in vars():
    update_group_vars(
        ugroup_name,
        nfs_disk_size=nfs_disk_size,
        nfs_device=nfs_device,
    )


=== 保存結果の確認 ==================================================================================================

保存した設定を表示します。確認してください。

In [ ]:
!cat group_vars/{ugroup_name}.yml

=== 必須パラメータの保存チェック ======================================================================================

次のセルがエラーになった場合は、必須パラメータが設定されていません。パラメータを確認してください。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
require_params = [
    'c_ip_address', 'compute_etc_hosts', 'compute_flavor',
    'compute_nodes', 'compute_use_gpu', 'master_flavor',
    'master_hostname', 'master_ipaddress', 'master_root_size',
    'ohpc_user', 'ohpc_user_prvkey', 'ohpc_user_pubkey',
    'slurm_conf', 'ssh_private_key_path', 'ssh_public_key_path',
    'ugroup_name', 'vault_path_munge_key', 'vc_provider',
]

for x in require_params:
    if x not in gvars:
        raise RuntimeError("ERROR: not set {}".format(x))

### OpenHPC 環境の構築

#### VCディスクの作成（based on 020-OpenHPCの起動.ipynb）

マスターノードのNFSサーバが公開するファイルを配置するためのディスクを作成します。

> 「3 パラメータ設定（based on 010-パラメータ設定.ipynb）」でVCディスクを作成しない設定にした場合は、この節を実行してもVCディスクは作成されません。

![VCディスク](images/ohpc-007.png)

=== VC ディスクの UnitGroup の作成 ===================================================================================

In [ ]:
if 'nfs_disk_size' in gvars:
    ug_disk = vcp.create_ugroup(f'{ugroup_name}_disk', ugroup_type='storage')

=== VC ディスクの作成 ===============================================================================================

In [ ]:
if 'nfs_disk_size' in gvars:
    provider = gvars['vc_provider']
    spec_disk = vcp.get_spec(f'{provider}_disk', 'small')
    if gvars['vc_provider'] == 'azure':
        spec_disk.disk_size_gb = gvars['nfs_disk_size']
    if gvars['vc_provider'] == 'oracle':
        spec_disk.size_in_gbs = gvars['nfs_disk_size']
    else:
        spec_disk.size = gvars['nfs_disk_size']
    unit_disk = ug_disk.create_unit('nfs', spec_disk)

UnitGroupに作成したVCディスクの一覧を表示します。

In [ ]:
from IPython.display import display
if 'unit_disk' in vars() and unit_disk:
    display(unit_disk.df_nodes())

#### VCノードの起動（based on 020-OpenHPCの起動.ipynb）

マスターノード、計算ノードとして利用するVCノードを起動します。

=== VC ノードの UnitGroup の作成 ====================================================================================

In [ ]:
ug = vcp.create_ugroup(ugroup_name)
vcp.df_ugroups()

=== マスターノードの起動 =============================================================================================

マスターノードとして利用するVCノードを起動します。

![マスターノード](images/ohpc-008.png)

マスターノード資源の spec の設定

In [ ]:
spec_master = vcp.get_spec(gvars['vc_provider'], gvars['master_flavor'])
spec_master.image = 'harbor.vcloud.nii.ac.jp/vcp/openhpc:master-2.4'
spec_master.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]
spec_master.ip_addresses = [gvars['master_ipaddress']]
if 'ug_disk' in vars() and ug_disk:
    spec_master.disks = ug_disk.find_nodes()
else:
    spec_master.params_v.append('/srv/OpenHPC:/exports')
if gvars['vc_provider'] == 'aws':
    spec_master.volume_size = gvars['master_root_size']
    if 'master_instance_type' in gvars:
        spec_master.instance_type = gvars['master_instance_type']
elif gvars['vc_provider'] == 'azure':
    spec_master.disk_size_gb = gvars['master_root_size']
    if 'master_instance_type' in gvars:
        spec_master.vm_size = gvars['master_instance_type']
elif gvars['vc_provider'] == 'oracle':
    spec_master.boot_volume_size_in_gbs = gvars['master_root_size']
    if 'master_instance_type' in gvars:
        spec_master.shape = gvars['master_instance_type']
spec_master.set_ssh_pubkey(gvars['ssh_public_key_path'])

ベースコンテナの環境変数の設定

* `SLURM_NODE_PARAMS`
    - slurm.conf に追加する計算ノードのパラメータ
* `MUNGE_KEY`
    - VCノードの `munge.key` の内容
* `NFS_DEV`
    - NFS用ディスクのデバイス名
    
さらに各ノードに対応するエントリを `/etc/hosts` に追加するためのパラメータを設定します。
> `spec` に設定した値を表示すると、その内容には **秘匿情報** である `munge.key` の内容を含んでいます。扱いには **注意** してください。

In [ ]:
%run scripts/utils_addition.py

spec_master.params_e.extend([
    f'SLURM_NODE_PARAMS={spec_env_slurm_conf(gvars)}',
    f'MUNGE_KEY={spec_env_munge_key(gvars, vcp, vcc_access_token)}',
])
if 'nfs_device' in gvars:
    spec_master.params_e.append(f'NFS_DEV={gvars["nfs_device"]}')
spec_master.hostname = gvars["master_hostname"]
spec_master.add_host = spec_add_host_list(gvars)

# print(spec_master)

VC ノードの起動（AWS の場合、およそ10分かかります）

In [ ]:
unit_master = ug.create_unit('master', spec_master)

VCノードの状態が `RUNNING` になっていることを確認します。VCノードの起動に失敗して`RUNNING`以外の状態になっている場合、次のセルがエラーとなります。

In [ ]:
rcount = 10             # retry count
while True:
    # マスターノードの状態取得
    unit_master = ug.get_unit('master')
    if any([node.state != 'RUNNING' for node in unit_master.find_nodes()]):
        !sleep 10
        rcount -= 1
        if rcount == 0:
            unit_master.df_nodes()
            raise RuntimeError('Error: cannot restart master node')
    else:
        break

`node_state`が`ERROR`となっている場合はノードの起動に失敗しています。`ug.cleanup()` を実行して VCノードを削除してください。その後、エラー原因を解消してから「[VCノードのUnitGroup作成](#VCノードのUnitGroup作成)」以下を `unfreeze` して再実行してください。

In [ ]:
unit_master.df_nodes()

=== NFS server の起動確認 ==========================================================================================

In [ ]:
addr = gvars['master_ipaddress']
ssh_opts = f'-o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -i {gvars["ssh_private_key_path"]}'
!ssh-keygen -R {addr}
!ssh {ssh_opts} vcp@{addr} systemctl status nfs-server

エラーになった場合は、起動したVCノードにて `journalctl` などを実行してエラーの原因を確認してください。原因となった事象を解消した後に次のセルのコメントを外してから実行し NFSサーバのサービスを再起動してください。

> よくある事例として、起動に時間がかかりタイムアウトすることでNFSサーバが起動しないことがあります。この場合は、単に次のセルのコメントを外してサービスの再起動を行うことで問題が解消します。

In [ ]:
# !ssh {ssh_opts} vcp@{addr} sudo systemctl restart nfs-server

サービスを再起動した場合は次のセルのコメントを外して、NFSサーバのサービスが起動したことを確認してください。

In [ ]:
# !ssh {ssh_opts} vcp@{addr} systemctl status nfs-server

NFSのエクスポート設定が１つ以上あることを確認します。次のセルを実行し、エラーとならないことを確認してください。

In [ ]:
# エクスポートの設定状況を表示する
!ssh {ssh_opts} vcp@{addr} "sudo /sbin/exportfs -v"
# エクスポートの設定項目が１つ以上あることをチェックする
!test $(ssh {ssh_opts} vcp@{addr} "sudo /sbin/exportfs -v" | wc -l) -gt 0 

上のセルがエラーとなる場合は、パラメータ設定にて指定した `nfs_device` の値が正しくないことが考えられます。次のセルのコメントを外してパラメータに設定した値と、起動したVCノードにアタッチされているブロックデバイス名が一致しているかを確認してください。正しく設定されている場合は、以下の表示例のように１行目と最後の行に表示されるブロックデバイス名（この例では`xvdf`） に同じ値が表示されます。

```
nfsdevice=/dev/xvdf
NAME    MAJ:MIN RM SIZE RO TYPE MOUNTPOINT
xvda    202:0    0  60G  0 disk 
└─xvda1 202:1    0  60G  0 part /etc/hosts
xvdf    202:80   0  64G  0 disk /exports
```

異なる値が表示されている場合は設定するパラメータを修正する必要があります。
「x920-OpenHPC環境の削除.ipynb」のNotebookで起動したVCノード、VCディスクを削除してから、再度「3. パラメータの設定」で、正しいブロックデバイス名を設定しなおし３章から再実行してください。

In [ ]:
# 設定されているパラメータ nfs_device の値を確認する
# print(f'nfsdevice={gvars["nfs_device"]}')

# VCノードのブロックデバイスを確認する
# !ssh {ssh_opts} vcp@{addr} lsblk 2> /dev/null

=== 計算ノードの起動 ================================================================================================

マスターノードとして利用するVCノードを起動します。

![計算ノード](images/ohpc-009.png)

計算ノード資源の spec の設定

In [ ]:
spec_compute = vcp.get_spec(gvars['vc_provider'], gvars['compute_flavor'])
spec_compute.image = 'harbor.vcloud.nii.ac.jp/vcp/openhpc:compute-2.4'
if gvars['compute_use_gpu']:
    spec_compute.gpus = 'all'
    spec_compute.image = 'harbor.vcloud.nii.ac.jp/vcp/openhpc:compute-gpu-2.4'
    if gvars['vc_provider'] != 'onpremises':
        spec_compute.cloud_image = 'niivcp-gpu-a-21.04.0-r1'
spec_compute.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
    '/opt/var/lib/docker:/var/lib/docker',
]
spec_compute.ip_addresses = [x for x in gvars['compute_etc_hosts'].keys()]
if 'compute_root_size' in gvars:
    if gvars['vc_provider'] == 'aws':
        spec_compute.volume_size = gvars['compute_root_size']
    elif gvars['vc_provider'] == 'azure':
        spec_compute.disk_size_gb = gvars['compute_root_size']
    elif gvars['vc_provider'] == 'oracle':
        spec_compute.boot_volume_size_in_gbs = gvars['compute_root_size']
if 'compute_instance_type' in gvars:
    if gvars['vc_provider'] == 'aws':
        spec_compute.instance_type = gvars['compute_instance_type']
    elif gvars['vc_provider'] == 'azure':
        spec_compute.vm_size = gvars['compute_instance_type']
    elif gvars['vc_provider'] == 'oracle':
        spec_compute.shape = gvars['compute_instance_type']
spec_compute.set_ssh_pubkey(gvars['ssh_public_key_path'])

ベースコンテナの環境変数に以下のパラメータを設定します。

* `MASTER_HOSTNAME`
    - マスターノードのホスト名
* `SLURM_NODE_PARAMS`
    - slurm.conf に追加する計算ノードのパラメータ
* `MUNGE_KEY`
    - VCノードの `munge.key` の内容
    
さらに各ノードに対応するエントリを `/etc/hosts` に追加するためのパラメータを設定します。

In [ ]:
%run scripts/utils_addition.py

spec_compute.params_e.extend([
    f'MASTER_HOSTNAME={gvars["master_hostname"]}',
    f'SLURM_NODE_PARAMS={spec_env_slurm_conf(gvars)}',
    f'MUNGE_KEY={spec_env_munge_key(gvars, vcp, vcc_access_token)}',
])
spec_compute.add_host = spec_add_host_list(gvars)

次のセルで `spec` に設定した値を表示すると、その内容には **秘匿情報** である `munge.key` の内容を含んでいます。扱いには **注意** してください。

In [ ]:
# print(spec_compute)

VC ノードを起動します（AWSで起動するにはおよそ２分かかります）。

In [ ]:
unit_compute = ug.create_unit('compute', spec_compute)

VCノードの状態が `RUNNING` になっていることを確認します。

> VCノードの起動に失敗して`RUNNING`以外の状態になっている場合は次のセルを実行するとエラーになります。エラーになった場合は、`ug.cleanup()` を実行して VCノードを削除してください。その後、「[VCノードのUnitGroup作成](#VCノードのUnitGroup作成)」以下を `unfreeze` してから再実行してください。

In [ ]:
unit_compute = ug.get_unit('compute')
if any([node.state != 'RUNNING' for node in unit_compute.find_nodes()]):
    raise RuntimeError('ERROR: not running')

起動したVCノードの一覧を表示します。

In [ ]:
unit_compute.df_nodes()

#### Ansibleの設定（based on 020-OpenHPCの起動.ipynb）

VCノードをAnsibleで操作するための設定を行います。

=== known_hosts の更新 =============================================================================================

VCノードをAnsibleで操作するための設定を行います。

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:

from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in ug.find_ip_addresses():
    !ssh-keygen -R {addr}
#    check_update_known_hosts(addr)
#    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts
    !ssh-keyscan {addr} >> ~/.ssh/known_hosts

```
★★★　注意　★★★

上のセルでエラーが発生した場合、次のセルのコメントを外して（行頭の'#'を削除）実行してから続けてください。
```

In [ ]:
#!ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

=== ansible イベントリへの登録 =======================================================================================

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
import yaml

inventory = {
    'all': {
        'children': {
            ug.name: {
                'children': dict([
                    (
                        f'{ug.name}_{unit.name}',
                        {
                            'hosts': dict([
                                (ip, {})
                                for ip in unit.find_ip_addresses()
                            ])
                        }
                    )
                    for unit in ug.find_units()]),
                'vars': {
                    'ansible_user': 'vcp',
                    'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
                }
            }
        }
    }
}

with open('inventory.yml', 'w') as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)
    
!cat inventory.yml

先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。

> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
from pathlib import Path

inventory_path = Path('./inventory.yml').resolve()
ansible_cfg = Path('./ansible.cfg').resolve()
with ansible_cfg.open(mode='w') as f:
    f.write(f'''
[defaults]
inventory = {inventory_path}
deprecation_warnings=False
remote_tmp = /var/tmp
timeout = 30
''')
ansible_cfg.parent.chmod(0o755)

!cat ./ansible.cfg

VCノードに対して Ansible で接続できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

正常に接続できると以下のように表示されます。

```
XXX.XXX.XXX.XXX | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
```

In [ ]:
!ansible {ugroup_name} -b -a 'whoami'

#### Slurmの状態確認（based on 020-OpenHPCの起動.ipynb）

起動したVCノードで実行されているSlurmの状態を確認します。

=== バッチジョブ投入の準備 ===========================================================================================

In [ ]:
# バッチジョブ投入ユティリティ(slurm 用)
def qsub(job):
    jobname = f'\"sbatch {job}\"'
    bs_state = !ansible {ugroup_name}_master -a {jobname}
    bs_state = bs_state[len(bs_state)-1]
    print(bs_state)

    if bs_state.split()[0] == 'Submitted':
        return bs_state.split()[3]
    else:
        raise RuntimeError('ERROR: job submission error')

        
def qstat(bs_jobid):
    print('job ' + bs_jobid + ':', end = ' ')
    while True:
        bs_state = !ansible {ugroup_name}_master -a 'squeue --noheader --jobs={bs_jobid}'
        bs_state = bs_state[len(bs_state)-1]
        if bs_state == '':
            break
        elif 'error' in bs_state:
            raise RuntimeError(bs_state)
        else:
            print("", end = '.')
            !sleep 5
    print(' finished')

=== ノードの状態を確認する ===========================================================================================

Slurm の`STATE`を確認し、 `down` となっている場合は状態をアップデートします。

In [ ]:
# ★★★ 自動リカバリのためオリジナルを改造 ★★★

count = 10
while True:
    bs_state = !ansible {ugroup_name}_master -a sinfo
    bs_state = bs_state[len(bs_state)-1]
    print(bs_state)
    
    if bs_state.split()[4] == 'idle':
        break
    else:
        for x in gvars['compute_etc_hosts'].values():
            !ansible {ugroup_name}_master -b -a 'scontrol update nodename={x} state=resume' || true
        !sleep 5
        
    count -= 1
    if count == 0:
        raise RuntimeError('Error: Slurm is down')

=== 動作確認のためジョブを実行する =====================================================================================

ジョブを実行できることを確認します。

`srun`コマンドで、`hostname` コマンドを計算ノードで実行します。

以下のような実行結果が表示されます（ホストの表示順序は入れ替わることがあります）。

```
0: c1

```

In [ ]:
!ansible {ugroup_name}_master -a 'srun -l -N {gvars["compute_nodes"]} hostname'

次に、サンプルの `hello.c` をコンパイルして `sbatch` で実行します。

In [ ]:
!ansible {ugroup_name}_master -a 'mpicc -O3 -o hello /opt/ohpc/pub/examples/mpi/hello.c'
!ansible {ugroup_name}_master -a 'ls -l hello'

ジョブスクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as work_dir:
    job_file = Path(work_dir) / 'hello.job'
    with job_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash

#SBATCH -J hello               # Job name
#SBATCH -o hello.%j.out         # Name of stdout output file (%j expands to jobId)
#SBATCH -N {gvars["compute_nodes"]}                  # Total number of nodes requested
#SBATCH -n {gvars["compute_nodes"]}                  # Total number of mpi tasks requested
#SBATCH -t 00:01:00           # Run time (hh:mm:ss)

# Launch MPI-based executable
prun ./hello
''')
    !cat {job_file}
    !ansible {ugroup_name}_master -m copy -a 'src={str(job_file)} dest=.'

ジョブを投入し、実行終了を待ちます。

In [ ]:
# ★★★ ジョブ終了確認自動化のためオリジナルを改造 ★★★
jobid = qsub('hello.job')
qstat(jobid)

ジョブの出力結果を表示してみます。ジョブの実行が成功していると以下のような出力結果が得られます。

```
[prun] Master compute host = c1
[prun] Resource manager = slurm
[prun] Launch cmd = mpirun ./hello (family=openmpi4)

 Hello, world (1 procs total)
    --> Process #   0 of   2 is alive. -> c1

```

In [ ]:
!ansible {ugroup_name}_master -m shell -a 'cat hello.*.out'

####  slurm 設定ファイルの編集（based on 032-設定ファイルの編集-GRESの登録.ipynb）

=== slurm.conf の編集 ==============================================================================================

★★★ slurm.conf 編集を自動化 ★★★★★★ 

Slurmの設定ファイル`slurm.conf`を編集します。

> 設定ファイルの記述方法の詳細については [slurm.conf(5)](https://slurm.schedmd.com/slurm.conf.html)を参照してください。

次のセルを実行すると、以下の手順が実行されます。

1. マスターノードの設定ファイル(slurm.conf)をローカル環境に取得する
1. 計算ノードのGPUを含めた詳細情報を取得する
1. ローカル環境に取得したslurm.confを計算ノードの情報で編集する

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/slurm.conf`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/slurm.conf.orig`

In [ ]:
# ★★★ slurm.conf 編集を自動化 ★★★
%run scripts/edit_conf.py

# slurm.conf バックアップ
slurm_conf_path = '/etc/slurm/slurm.conf'
local_path = download_conf_file(slurm_conf_path, ugroup_name, 'master')
org = local_path.parent / (local_path.name + '.orig')
shutil.copy2(local_path, org)

# 計算ノード情報生成
cr_info_rep = 'NodeName='
cr_info_rep += f'{c_hostname_prefix}[1-{compute_nodes}]' if compute_nodes > 1 else f'{c_hostname_prefix}1'
cr_info_rep += ' Gres=gpu:1'
cr_info = !ansible {ugroup_name}_compute -a '/usr/sbin/slurmd -C'
for x in range(1, len(cr_info), 3):
    if cr_info[x][0:9] == 'NodeName=':
        for y in range(1, len(cr_info[x].split())):
            cr_info_rep += ' ' + cr_info[x].split()[y]
        cr_info_rep += ' State=UNKNOWN\nGresTypes=gpu\n'
        break

# 編集(NodeName の置き換え)
with open(Path(org)) as f:
    content = f.readlines()
for x in range(len(content)):
    if content[x][0:9] == 'NodeName=':
        content[x] = cr_info_rep
        break
with open(Path(local_path), mode='w') as f:
    f.writelines(content)

参考：計算ノードの GPU 情報

In [ ]:
!ansible {ugroup_name}_compute -a 'nvidia-smi -L'

編集前後の差分を表示します。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(slurm_conf_path, ugroup_name, 'master')

=== gres.conf の編集 ===============================================================================================

★★★ gres.conf 編集を自動化 ★★★★★★ 

`/dev/nvidia0`などのデバイスファイルとの関連付けを行うために `gres.conf` を編集します。

In [ ]:
# ★★★ gres.conf 編集を自動化 ★★★
%run scripts/edit_conf.py

# gres.conf バックアップ
gres_conf_path = '/etc/slurm/gres.conf'
!ansible {ugroup_name}_master -m file -b -a 'path={gres_conf_path} state=touch'
local_path = download_conf_file(gres_conf_path, ugroup_name, 'master')
org = local_path.parent / (local_path.name + '.orig')
shutil.copy2(local_path, org)

# gpu 情報の追加
!ansible {ugroup_name}_master -m file -b \
    -a 'path={gres_conf_path} state=touch'
with open(local_path, mode='a') as f:
    f.write('Name=gpu File=/dev/nvidia0\n')

`gres.conf`の設定内容の詳細については [gres.conf(5)](https://slurm.schedmd.com/gres.conf.html) を参照してください。

編集前後の差分を表示します。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(gres_conf_path, ugroup_name, 'master')

=== 編集した設定ファイルの反映 ========================================================================================

編集したファイルをVCノードに配置して、設定ファイルの変更内容をコンテナに反映させます。

まず、変更前のノードの状態を確認します。

In [ ]:
!ansible {ugroup_name}_master -a 'scontrol show node'

設定ファイルの変更を行います。

次のセルを実行すると、`slurm.conf`, `gres.conf` のそれぞれの設定ファイルについて以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイルを各VCノードに配置する

In [ ]:
%run scripts/edit_conf.py
upload_conf(slurm_conf_path, ugroup_name)
upload_conf(gres_conf_path, ugroup_name)

変更後の設定を反映させるためにマスターノード、計算ノードの SLURMデーモンを再起動します。

In [ ]:
!ansible {ugroup_name}_master -b -a 'systemctl restart slurmctld'
!ansible {ugroup_name}_compute -b -a 'systemctl restart slurmd'

ノードの状態を確認し、drain の場合各ノードの状態をアップデートします。

In [ ]:
# ★★★ STATE = drain 時の自動リカバリのためにオリジナルを改造 ★★★
%run scripts/group.py

count = 5
while True:
    bs_state = !ansible {ugroup_name}_master -a sinfo
    bs_state = bs_state[len(bs_state)-1]
    print(bs_state)
    
    if bs_state.split()[4] != 'drain':
        break
    else:
        gvars = load_group_vars(ugroup_name)
        for x in gvars['compute_etc_hosts'].values():
            !ansible {ugroup_name}_master -b -a 'scontrol update nodename={x} state=resume' || true

        !sleep 5           
            
    count -= 1
    if count == 0:
        raise RuntimeError('Error: node drain')

各ノードのGRESの状況を表示します。

In [ ]:
!ansible {ugroup_name}_master -a 'sinfo --Node --Format=NodeHost,Gres,GresUsed'

ただしく設定変更が行われていると`GRES`の欄にGPUが表示されます。`GRES`の列に GPU が表示されていることを確認してください。
```
HOSTNAMES           GRES             GRES_USED           
c1                  gpu:1            gpu:0

```



ノード状態の詳細を表示します。

In [ ]:
!ansible {ugroup_name}_master -a 'scontrol show node'

表示されたノードの状態が正しくない場合は「4. 設定ファイルの編集」以降のセルを unfreeze してから再度設定ファイルの変更を行ってください。

#### Docker Engineのインストール（based on 071-DockerEngineのインストール.ipynb）

[Install Docker Engine on CentOS](https://docs.docker.com/engine/install/centos/)の手順に従い Docekr Engineを計算ノードにインストールします。

=== Docker レポジトリの追加 =========================================================================================

Docker のレポジトリを追加します。

In [ ]:
!ansible {ugroup_name}_compute -b -m yum -a 'name=yum-utils'
!ansible {ugroup_name}_compute -b -a 'yum-config-manager --add-repo \
    https://download.docker.com/linux/centos/docker-ce.repo'

=== Docker エンジンのインストールと起動 ===============================================================================

Docker Engine をインストールします。

In [ ]:
!ansible {ugroup_name}_compute -b -m yum -a 'name=docker-ce,docker-ce-cli,containerd.io'

Docker Engineのサービスを開始します。

In [ ]:
!ansible {ugroup_name}_compute -b -m systemd -a 'name=docker enabled=yes state=started'

Docker Engineが利用できることを確認するために `docker info`コマンドを実行します。

In [ ]:
!ansible {ugroup_name}_compute -a 'docker info'

コンテナを実行できることを確認するために`hello-world`コンテナを実行します。

In [ ]:
!ansible {ugroup_name}_compute -a 'docker run --rm hello-world'

#### NVIDIA Container Toolkit のインストール（based on 071-DockerEngineのインストール.ipynb）

=== レポジトリの追加 ================================================================================================

[Setting up NVIDIA Container Toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#id2)の手順に従いインストールを行います。

> 計算ノードでGPUが利用できる場合のみ NVIDIA Container Toolkitのインストールが必要となります。計算ノードでGPUが利用できない場合はこの節の実行は不要です。

レポジトリを追加します。

In [ ]:
!ansible {ugroup_name}_compute -b -m shell -a \
    'distribution=$(. /etc/os-release;echo rhel$VERSION_ID) \
    && curl -s -L https://nvidia.github.io/nvidia-docker/$distribution/nvidia-docker.repo | tee /etc/yum.repos.d/nvidia-docker.repo'

=== nvidia-docker2 パッケージのインストール ===========================================================================

In [ ]:
!ansible {ugroup_name}_compute -b -m dnf -a \
    'name=nvidia-docker2 update_cache=yes'

変更を反映するために Docker Engine を再起動します。

In [ ]:
!ansible {ugroup_name}_compute -b -a 'systemctl restart docker'

変更が反映されたことを確認するためにDockerの情報を表示させます。`Server`の `Runtimes`に`nvidia`が追加されたことを確認してください。
表示例を以下に示します。

```
Client:
 Context:    default
 Debug Mode: false
 Plugins:
  app: Docker App (Docker Inc., v0.9.1-beta3)
  buildx: Build with BuildKit (Docker Inc., v0.5.1-docker)
  scan: Docker Scan (Docker Inc.)

Server:
(中略)
 Runtimes: io.containerd.runc.v2 io.containerd.runtime.v1.linux nvidia runc
 Default Runtime: runc
(以降略) 
```

In [ ]:
!ansible {ugroup_name}_compute -b -a 'docker info'

#### ユーザの追加（based on 051-ユーザの追加.ipynb）

=== ユーザの追加 ====================================================================================================

OpenHPC環境にユーザを追加します。

In [ ]:
!ansible {ugroup_name}_master -b -m user -a 'name={ohpc_user}'
!ansible {ugroup_name}_compute -b -m user -a 'name={ohpc_user}'

作成したユーザのSSHの公開鍵を `$HOME/.ssh/authorized_keys` に追加します。

In [ ]:

import os
pubkey = os.path.expanduser(ohpc_user_pubkey)
with open(pubkey) as f:
    ssh_public_key = f.read()
print(ssh_public_key)

入力されたSSHの公開鍵を登録します。

In [ ]:
!ansible {ugroup_name}_master -b -m authorized_key -a 'user={ohpc_user} key="{ssh_public_key}"'

登録したユーザでログインして、コマンドを実行しエラーにならないことを確認します。

In [ ]:
!ssh -i {ohpc_user_prvkey} {ohpc_user}@{master_ipaddress} hostname

=== グループの追加 ==================================================================================================

ユーザをプライマリグループ(メイングループ)以外のグループに登録します。

> 登録ユーザに docker コマンドの実行を許可する場合は `docker` グループへの登録が必要となります。

In [ ]:

groups = [
    'docker',
]

ユーザを指定したグループに追加します。

In [ ]:
if len(groups) > 0:
    !ansible {ugroup_name}_master -b -m user -a 'name={ohpc_user} append=yes groups={",".join(groups)}'
    !ansible {ugroup_name}_compute -b -m user -a 'name={ohpc_user} append=yes groups={",".join(groups)}'

ユーザがグループに登録されたことを確認します。

In [ ]:
!ssh -i {ohpc_user_prvkey} {ohpc_user}@{master_ipaddress} id

### TensorFlow環境の構築

構築した OpenHPC 環境で NGC Catalog の [TensorFlowコンテナ](https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow) を実行します。

#### 準備（based on 152-NGCのコンテナ実行-TensorFlow.ipynb）

In [ ]:
# ユーザ名とホスト名
target = f'{ohpc_user}@{master_ipaddress}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ohpc_user_prvkey}' if 'ohpc_user_prvkey' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
# !ssh {ssh_opts} {target} hostname

Dockerを利用できることを確認します。

In [ ]:
!ssh {ssh_opts} {target} 'srun -N 1 docker info'

上のセルがエラーになった場合は Docker Engineがインストールされていないか docker を実行する権限が与えられていません。

以下のようなエラーが表示された場合は Docker Engine がインストールされていません。「4.2.6  Docker Engineのインストール（based on 071-DockerEngineのインストール.ipynb）」を参考にDocker Engineを利用できるようにしてください。
```
slurmstepd: error: execve(): docker: No such file or directory
```

また、以下のようなエラーが表示された場合は docker を実行する権限が与えられていません。「4.2.8  ユーザの追加（based on 051-ユーザの追加.ipynb）」の「グループの追加」などを参考に`docker`グループへの追加を行ってください。
```
ERROR: Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: 
```

SlurmのGeneric Resource(GRES)としてGPUが登録されていることを確認します。

In [ ]:
!ssh {ssh_opts} {target} sinfo -N --Format=NodeHost,Gres | grep -w gpu

上のセルがエラーになった場合はSlurmにGPUがGRESとして登録されていません。「4.2.5  slurm 設定ファイルの編集（based on 032-設定ファイルの編集-GRESの登録.ipynb）」を参考にGRESの設定をしてください。

#### TensorFlow コンテナイメージの取得（based on 152-NGCのコンテナ実行-TensorFlow.ipynb）

NGCカタログから[TensorFlow](https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow)のコンテナイメージを取得します。

In [ ]:
ngc_version = '22.01'
docker_image = f'nvcr.io/nvidia/tensorflow:{ngc_version}-tf2-py3'
!ssh {ssh_opts} {target} srun -N {compute_nodes} docker pull {docker_image}

ユーザを追加したコンテナイメージを作成します。

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

!ssh {ssh_opts} {target} mkdir -p tensorflow-img
out = !ssh {ssh_opts} {target} id -u
uid = out[0]
with TemporaryDirectory() as workdir:
    dockerfile = Path(workdir) / 'Dockerfile'
    with dockerfile.open(mode='w') as f:
        f.write(f'''
FROM {docker_image}

ARG USER={ohpc_user}
RUN useradd -m ${{USER}} -u {uid}
WORKDIR /home/${{USER}}
USER {ohpc_user}
''')
    !cat {dockerfile}
    !scp {ssh_opts} {dockerfile} {target}:tensorflow-img

!ssh {ssh_opts} {target} srun -N {compute_nodes} docker build -t tensorflow-{ohpc_user} tensorflow-img

#### 動作確認（based on 152-NGCのコンテナ実行-TensorFlow.ipynb）

手書き数字認識データ MNIST を学習させることで、TensorFlow の正常動作を確認します。

=== 準備 ==========================================================================================================

MNISTのデータやスクリプトを準備します。

まず、データやスクリプトを配置するディレクトリを作成します。

In [ ]:
work_dir = 'tensorflow'
!ssh {ssh_opts} {target} mkdir -p {work_dir}

データをダウンロードするためのスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/download_mnist.py {target}:{work_dir}

TensorFlowの[Quickstart for beginners](https://www.tensorflow.org/tutorials/quickstart/beginner)で実行しているのと同等のスクリプトをOpenHPC環境に配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/mnist_classify.py {target}:{work_dir}

=== コンテナからGPUが利用できることの確認 ==============================================================================

Singularityで実行したコンテナ環境からGPUを利用できることを確認します。

GPUを利用できるかをチェックするスクリプトを配置します。

In [ ]:
!scp {ssh_opts} template/tensorflow/check_gpu.py {target}:{work_dir}

コンテナを実行できるようにするためにホームディレクトリのパーミッションを変更します。

In [ ]:
!ssh {ssh_opts} {target} chmod 755 .

GPU利用の可否をチェックするスクリプトを実行します。エラーとならないことを確認してください。

In [ ]:
!ssh {ssh_opts} {target} srun -N {compute_nodes} --gpus-per-node=1 \
    docker run --gpus all --ipc=host -v /home/{ohpc_user}:/home/{ohpc_user} --rm tensorflow-{ohpc_user} \
    python {work_dir}/check_gpu.py

=== ジョブの実行 ====================================================================================================

MNISTのスクリプトをSlurmのジョブとして実行します。

ジョブの実行スクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as workdir:
    batch_file = Path(workdir) / 'tensorflow_mnist_docker.job'
    with batch_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash
        
#SBATCH -J tensorflow-mnist-docker         # create a short name for your job
#SBATCH -o tensorflow-mnist-docker.%j.out  # Name of stdout output file (%j expands to jobId)
#SBATCH -N 1                               # Total number of nodes requested
#SBATCH -n 1                               # Total number of across all nodes
#SBATCH --gres=gpu:1                       # number of gpus per node
#SBATCH -t 00:10:00                        # Run time (hh:mm:ss)

docker run -td --gpus all -v /home/{ohpc_user}:/home/{ohpc_user} --rm --ipc=host --net=host --name tensorflow-{ohpc_user} tensorflow-{ohpc_user}
docker exec -t -u {ohpc_user} -w $HOME/{work_dir} tensorflow-{ohpc_user} python3 download_mnist.py
docker exec -t -u {ohpc_user} -w $HOME/{work_dir} tensorflow-{ohpc_user} python3 mnist_classify.py
docker stop tensorflow-{ohpc_user}
''')
    !cat {batch_file}
    !scp {ssh_opts} {str(batch_file)} {target}:{work_dir}

ジョブを実行する前のキューの状態を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

ノードのGPU利用状況を確認します。`GRES_USED` の欄でノードの GPU 利用状況を確認してください。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブを実行します。

In [ ]:
# ★★★ ジョブ終了確認自動化のためオリジナルを改造 （slurm 専用）★★★
bs_state = !ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && sbatch tensorflow_mnist_docker.job'"
bs_state = bs_state[len(bs_state)-1]
print(bs_state)

if bs_state.split()[0] == 'Submitted':
    bs_jobid = bs_state.split()[3]
else:
    raise RuntimeError('ERROR: job submission error')

ノードのGPU利用状況を確認します。`GRES_USED` の欄でノードの GPU 利用状況を確認してください。

In [ ]:
!ssh {ssh_opts} {target} sinfo --Node --Format=NodeHost,Gres,GresUsed

ジョブの完了を待ちます。数分かかります。ジョブ実行中のCPU, メモリ, GPUなどの利用状況は VCC の Grafana で確認することができます。

In [ ]:
# ★★★ ジョブ終了待ち（オリジナルに追加。slurm 専用）★★★

print('job ' + bs_jobid + ':', end = ' ')

while True:
    bs_state = !ssh {ssh_opts} {target} bash -l -c 'squeue --noheader --jobs={bs_jobid}'
    bs_state = bs_state[len(bs_state)-1]
    if bs_state.split()[0] != bs_jobid:
        break
    else:
        print("", end = '.')
        !sleep 5
    
print(' finished')

ジョブの出力結果を確認します。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c \
    "'cd {work_dir} && tail tensorflow-mnist*.out'"

ジョブの実行が完了していれば出力結果の末尾に以下のような内容が表示されます。

```
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1098 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0891 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0748 - accuracy: 0.9755
313/313 - 0s - loss: 0.0839 - accuracy: 0.9748

Test accuracy: 0.9747999906539917
```

=== 構築終了日時 ====================================================================================================

In [ ]:
!date